In [3]:
import threading

threading.active_count() #有幾個active的thread

5

In [4]:
threading.enumerate() #這些thread分別是什麼
# [<_MainThread(MainThread, started 140736011932608)>, <Thread(SockThread, started daemon 123145376751616)>, ...]

[<_MainThread(MainThread, started 140736680096704)>,
 <Thread(Thread-2, started daemon 123145392816128)>,
 <Heartbeat(Thread-3, started daemon 123145398071296)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145404399616)>,
 <ParentPollerUnix(Thread-1, started daemon 123145409654784)>]

In [6]:
threading.current_thread() #目前正在運行的thread
# <_MainThread(MainThread, started 140736011932608)>

<_MainThread(MainThread, started 140736680096704)>

In [7]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

def main():
    thread = threading.Thread(target=thread_job)   #添加一個thread
    thread.start()  # 執行thread
    
if __name__ == '__main__':
    main()

This is a thread of <Thread(Thread-4, started 123145414909952)>


In [10]:
import threading
import time

def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1) # 任务间隔0.1s
    print("T1 finish\n")

added_thread = threading.Thread(target=thread_job, name='T1')
added_thread.start()
# added_thread.join() #join後 要等待 added_thread 結束後,才接下去作另一個功能
print("all done\n")

T1 start

all done

T1 finish



# join()
程式在跑的時候主要是運行 MainThread

$ threading.current_thread() #目前正在運行的thread

再添加thread的時候,所有thread會同步運行,

而當有thread使用 join() 則是把這個thread join 到 MainThread,

以至於 MainThread (script中沒有被寫進threading.Thread(target=ＸＸＸ)的程式碼)

必須等待此join進來的thread執行完畢後再繼續往下運行！

ps. 程式運行 MainThread 所以當有新thread join時, 其他已經啟動的thread(除MainThread), 不會等待join的thread,
但是！ 要是新的 thread 還沒被 MainThread 執行到啟動, 則還是要一起等待先前 join 進來的 thread 結束, MainThread 才會執行, 新的 thread 也才會被啟動與 MainThread 一起執行.

In [25]:
def T1_job():
    print(f"T1 start {threading.current_thread()}\n")
    for i in range(10):
        time.sleep(0.1)
    print(f"T1 finish {threading.current_thread()}\n")

def T2_job():
    print(f"T2 start {threading.current_thread()}\n")
    time.sleep(5)
    print(f"T2 finish {threading.current_thread()}\n")

    
print(threading.current_thread())
thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 开启T1
# thread_1.join() 
thread_2.start() # 开启T2
# thread_2.join()
# print(threading.enumerate())
print(f"all done {threading.current_thread()}\n")

<_MainThread(MainThread, started 140736680096704)>
T1 start <Thread(T1, started 123145414909952)>

T2 start <Thread(T2, started 123145420165120)>

all done <_MainThread(MainThread, started 140736680096704)>

T1 finish <Thread(T1, started 123145414909952)>

T2 finish <Thread(T2, started 123145420165120)>



In [26]:
# 添加的thread無法return值, 所以如果需要返回數值回主線程運算, 需要儲存隊列
import threading
import time

from queue import Queue

def job(l,q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)

def multithreading():
    q = Queue()
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    
    for i in range(4):  # 運行四個線程 來計算data的四個元素
        t = threading.Thread(target=job, args=(data[i], q))
        t.start()
        threads.append(t)
        
    for thread in threads:
        thread.join()  # 讓主線程等待所有 thread完成 再繼續往下做
        
    results = []
    for _ in range(4):
          results.append(q.get())
    print(results)

if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


In [29]:
# GIL (global interpreter lock),多個thread其實一次只執行一個,只是互相切換
# 原理是一次只運行一個 thread, 並且用GIL讓程序指運行此thread, 然後在 I/O 的時候把運行權力交給下一個thread,
# 所以雖然用了多線程,但實際上是多個線程輪流在I/O時切換工作, “減少需要讀寫所花費的時間”
# 所以還是要看情況, multithreading 不一定能夠提高處理效率！
# 如果你的程序大部分只會涉及到I/O，比如網絡交互，那麼使用多線程就很合適，因為它們大部分時間都在等待。
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

1999998000000
normal:  0.06835818290710449
1999998000000
multithreading:  0.06331515312194824


In [30]:
#當兩個thread需要用到共享的內存,需要lock功能
#若不用lock,此案例會job1與job2同時運行,A會亂跳
import threading
def job1():
    global A, lock
    lock.acquire() #開始lock
    for i in range(10):
        A += 1
        print(f'job1={A} \n')
    lock.release() #結束lock

def job2():
    global A, lock
    lock.acquire()
    for i in range(10):
        A += 10
        print(f'job2={A} \n')
    lock.release()

if __name__ == '__main__':
    lock = threading.Lock()
    A = 0  #共享內存,全局變量
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()


job1=1 

job1=2 

job1=3 

job1=4 

job1=5 

job1=6 

job1=7 

job1=8 

job1=9 

job1=10 

job2=20 

job2=30 

job2=40 

job2=50 

job2=60 

job2=70 

job2=80 

job2=90 

job2=100 

job2=110 

